In [10]:
import numpy as np
import statsmodels.tsa.api as smt
import matplotlib.pyplot as plt
from aix360.datasets.SIF_dataset import DataSetTS
from aix360.algorithms.sif.SIF_NN import AllRNN, AllLSTM, AllAR
from aix360.algorithms.sif.SIF_utils import get_contaminate_series
import os
import pickle

#### model below can be changed to AllLSTM or AllRNN model defined in SIF_NN.py

In [11]:
# arma2 model
def get_model_train_ar2(data_sets, series, timesteps, w=None, gammas=None, num_train_steps=20000,
                        model_dir=None):
    initial_learning_rate = 0.01
    decay_epochs = [10000, 20000]
    batch_size = 20
    n_sample = series.shape[1] if len(series.shape) > 1 else 1

    # model can be changed to AllLSTM or AllRNN model defined in SIF_NN.py
    model = AllAR(
        time_steps=timesteps,
        x_dim=n_sample,
        y_dim=n_sample,
        share_param=True,
        batch_size=batch_size,
        time_series=series,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        damping=1e-3,
        decay_epochs=decay_epochs,
        mini_batch=False,
        train_dir='arma_output',
        log_dir='logs',
        model_name='ar_test',
        calc_hessian=False,
        w=w,
        gammas=gammas,
    )
    if model_dir is not None:
        print('Loading pre-trained model......')
        model.restore(model_dir)
    else:
        model.train(num_steps=num_train_steps, iter_to_switch_to_batch=10, iter_to_switch_to_sgd=10)
    return model

#### In default, we use the fast mode to accelerate the computation.


In [14]:
fast_mode = True

# parameters
timesteps = 2
np.random.seed(1)
n_sample = 1000
n_time_stamp = 100
gammas = np.arange(0.0, 0.09, 0.01)

#### Please ensure that data.pkl and model are downloaded in the following directories.

In [15]:
data_dir = '../../aix360/data/sif_data/data.pkl'
model_dir = '../../aix360/models/sif/ar2'


#### Skip generating the synthetic dataset and training the model if using fast-mode. In the fast mode, we directly load the saved dataset and pre-trained model

In [16]:
if fast_mode:
    assert os.path.exists(data_dir), "Could not find the data.pkl in {}".format(data_dir)
    # load time series from data.pkl file
    series = pickle.load(open(data_dir, "rb"))
    data_sets = DataSetTS(np.arange(len(series)), np.array(['Y']), None, None, None,
                          lag=timesteps).datasets_gen_rnn()
    # initialize and train the model which takes the clean time sequence as input and makes prediction
    model = get_model_train_ar2(data_sets, series, timesteps, gammas=gammas, model_dir=model_dir)
else:
    # ar and ma are two parameters controlling the synthetic time sequence data
    ar = np.r_[1, -np.array([0.7, -0.3])]
    ma = np.r_[1, -np.array([0.1])]

    # generate the core process or clean time sequence data
    series = [smt.arma_generate_sample(ar, ma, n_time_stamp) for i in range(n_sample)]
    series = np.vstack(series)
    pickle.dump(series, open(data_dir, "wb"))
    data_sets = DataSetTS(np.arange(len(series)), np.array(['Y']), None, None, None,
                          lag=timesteps).datasets_gen_rnn()
    # initialize and train the model which takes the clean time sequence as input and makes prediction
    model = get_model_train_ar2(data_sets, series, timesteps, gammas=gammas, model_dir=None)
    model.save(model_dir)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







Total number of parameters: 2
Loading pre-trained model......
INFO:tensorflow:Restoring parameters from ../../aix360/models/sif/ar2


#### y_contaminate is considered as the contaminating process in the experiment

In [17]:
y_contaminate = np.random.randn(n_sample)

#### Insert the contaminated data into the clean time sequence data

In [18]:
contaminated_series = get_contaminate_series(series, y_contaminate, data_sets.train.labels)

# plot contaminated series
plt.plot(contaminated_series)
plt.savefig('arma')
plt.close()


#### compute SIF value

In [19]:
# compute SIF value
model.update_configure(y_contaminate, gammas)
sif = model.get_sif(y_contaminate, timesteps, 1, None, 10, verbose=False)
print('SIF = {}'.format(sif))


Regression R-squared: 0.270422
Regression R-squared: 0.149079
54.085068464279175 s to compute if_v
0.0359044075012207 s to compute patchy pred gamma
0.06781840324401855 s to compute psi_y
SIF = 166.51340947947648
